In [ ]:
import numpy as np
import pandas as pd

import os

import torch
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn as nn 
from sklearn.model_selection import train_test_split
import warnings

In [ ]:
from google.colab import files
uploaded = files .upload()

import io

train = pd.read_csv(io.BytesIO(uploaded['train.csv']),dtype = np.float32)#Takes in the data

TypeError: ignored

In [ ]:
targets_numpy = train.label.values#splits the data into labels
features_numpy = train.loc[:,train.columns != "label"].values/255#splits the data into features
#test data is 20% of the data and the other 80% is used to train the model
features_train, features_test, targets_train, targets_test = train_test_split(features_numpy,
                                                                             targets_numpy,
                                                                             test_size = 0.2,
                                                                             random_state = 42) 

featuresTrain = torch.from_numpy(features_train) #create feature tensor for train set
targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor)#create target tensor for train set

featuresTest = torch.from_numpy(features_test)#create feature tensor for test set
targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor)#create target tensor for test set

batch_size = 100
num_epochs = 200
num_epochs = int(num_epochs)


train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)#Combines the two training datasets into an pytorch set?
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)#data loader
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=0)#Convolution 1
        self.relu1 = nn.ReLU()#Rectified linear unit funciotn elemet-wise

        self.maxpool1 = nn.MaxPool2d(kernel_size=2)#?

        self.cnn2 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()

        self.maxpool2 = nn.MaxPool2d(kernel_size=2)#?

        self.fc1 = nn.Linear(32 * 4 * 4, 10) 
    
    def forward(self, x):

        out = self.cnn1(x)
        out = self.relu1(out)

        out = self.maxpool1(out)

        out = self.cnn2(out)
        out = self.relu2(out)
        
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)

        out = self.fc1(out)
        
        return out

batch_size = 100
num_epochs = 10
num_epochs = int(num_epochs)

train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)#Combines the two training datasets into an pytorch set?
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)#dataloader
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

model = CNNModel()#this is the machine learning model that will be trained using the data

error = nn.CrossEntropyLoss()#determines loss in data?

learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)#optimizes the funtion

In [ ]:
count = 0
loss_list = []
iteration_list = []
accuracy_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        train = Variable(images.view(100,1,28,28))
        labels = Variable(labels)

        optimizer.zero_grad()#clears the gradients

        outputs = model(train)

        loss = error(outputs, labels)#calculates entropy loss

        loss.backward()#calculates the gradients

        optimizer.step()#Update all the parameters
        count += 1
        if count % 50 == 0:
            correct = 0#Calculate the accuracy of the model
            total = 0

            for images, labels in test_loader:#Goes through the dataset
                
                test = Variable(images.view(100,1,28,28))

                outputs = model(test)#tests the predictions and the actual value

                predicted = torch.max(outputs.data, 1)[1]#Gets the predicitions from the max value

                total += len(labels)#total num of labels
                
                correct += (predicted == labels).sum()#number of correctly predicted values
            
            accuracy = 100 * correct / float(total)

            loss_list.append(loss.data)#store the loss
            iteration_list.append(count)#store the interations
            accuracy_list.append(accuracy)#store the accuracy
            if count % 500 == 0:
                print('Iteration: {}  Accuracy: {} %'.format(count, accuracy))